In [1]:
import os
os.chdir("/home/ec2-user/SageMaker/foodi-ml/")

In [2]:
#!pip install -r requirements.txt

In [3]:
import os
import torch
from tqdm import tqdm

import params
from retrieval.train import train
from retrieval.utils import helper
from retrieval.model import loss
from retrieval.model.model import Retrieval
from retrieval.data.loaders import get_loaders
from retrieval.utils.logger import create_logger
from retrieval.utils.helper import load_model
from retrieval.utils.file_utils import load_yaml_opts, parse_loader_name

In [4]:
from addict import Dict

# Functions

In [5]:
def get_data_path(opt):
    if 'DATA_PATH' not in os.environ:
        if not opt.dataset.data_path:
            raise Exception('''
                DATA_PATH not specified.
                Please, run "$ export DATA_PATH=/path/to/dataset"
                or add path to yaml file
            ''')
        return opt.dataset.data_path
    else:
        return os.environ['DATA_PATH']

In [6]:
def get_tokenizers(train_loader):
    tokenizers = train_loader.dataset.tokenizer
    if type(tokenizers) != list:
        tokenizers = [tokenizers]
    return tokenizers

In [7]:
def set_criterion(opt, model):
    if 'name' in opt.criterion:
        logger.info(opt.criterion)
        multimodal_criterion = loss.get_loss(**opt.criterion)
        multilanguage_criterion = loss.get_loss(**opt.criterion)
    else:
        multimodal_criterion = loss.ContrastiveLoss(**opt.criterion)
        multilanguage_criterion = loss.ContrastiveLoss(**opt.ml_criterion)
    set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion)
    # return multimodal_criterion, multilanguage_criterion


def set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion):
    model.mm_criterion = multimodal_criterion
    model.ml_criterion = None
    if len(opt.dataset.adapt.data) > 0:
        model.ml_criterion = multilanguage_criterion

# Run

In [8]:
os.environ["DATA_PATH"] = "/home/ec2-user/SageMaker/data/"

In [9]:
options = "options/adapt/foodi-ml/i2t.yaml"

In [10]:
args = {
    "options": options,
}
args = Dict(args)
opt = load_yaml_opts(args.options)

In [11]:
logger = create_logger(level='debug' if opt.engine.debug else 'info')
#logger.info(f'Used args   : \n{args}')
#logger.info(f'Used options: \n{opt}')

In [12]:
# Get path of the data
data_path = get_data_path(opt)

In [13]:
# Get loaders
train_loader, val_loaders, adapt_loaders = get_loaders(data_path, args.local_rank, opt)

2021-08-16 18:53:50,726 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-16 18:53:50,727 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-16 18:53:50,727 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-16 18:53:50,771 - [INFO    ] - [FoodiML] Loaded 8011 images and 8011 annotations.
2021-08-16 18:53:50,775 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-16 18:53:50,776 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-16 18:53:50,776 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-16 18:53:50,813 - [INFO    ] - [FoodiML] Loaded 0 images and 0 annotations.
2021-08-16 18:53:50,814 - [INFO    ] - Adapt loaders: 0


In [14]:
tokenizers = get_tokenizers(train_loader)

In [15]:
model = Retrieval(**opt.model, tokenizers=tokenizers)

2021-08-16 18:53:55,925 - [INFO    ] - Image encoder created: ('resnet50',)
2021-08-16 18:53:56,032 - [INFO    ] - Text encoder created: gru_glove
2021-08-16 18:53:56,033 - [INFO    ] - Created similarity: Cosine()
2021-08-16 18:53:58,946 - [INFO    ] - Setting devices: img: cuda,txt: cuda, loss: cuda
2021-08-16 18:53:58,946 - [INFO    ] - Using similarity: ('cosine',)


In [17]:
print_fn = (lambda x: x) if not model.master else tqdm.write

In [18]:
# Trainer
trainer = train.Trainer(
        model=model,
        args=opt,
        sysoutlog=print_fn,
        path=opt.exp.outpath,
        world_size=1 # TODO
)

In [19]:
trainer.setup_optim(
        lr=opt.optimizer.lr,
        lr_scheduler=opt.optimizer.lr_scheduler,
        clip_grad=opt.optimizer.grad_clip,
        log_grad_norm=False,
        log_histograms=False,
        optimizer=opt.optimizer,
        freeze_modules=opt.model.freeze_modules
)

2021-08-16 18:54:23,480 - [INFO    ] - lr 0.001
2021-08-16 18:54:23,480 - [INFO    ] - [0.5, 2.0, 4000]
2021-08-16 18:54:23,481 - [INFO    ] - [10000, 20000, 3000]


Freezing model.txt_enc.embed.glove
lr: 0.001, #layers: 170, #params: 36,342,452
Total Params: 38,846,552, 


In [20]:
trainer.fit(
        train_loader=train_loader,
        valid_loaders=val_loaders,
        lang_loaders=adapt_loaders,
        nb_epochs=opt.engine.nb_epochs,
        valid_interval=opt.engine.valid_interval,
        log_interval=opt.engine.print_freq
)

logs/foodi-ml/adapt_i2t/ already exists! Do you want to rewrite it? [y/n]  y


Steps :   0%|          | 0/802 [00:00<?, ?it/s]
                                               


RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D